<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Creating-duration-in-minutes-column" data-toc-modified-id="Creating-duration-in-minutes-column-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Creating duration in minutes column</a></span><ul class="toc-item"><li><span><a href="#Checking-for-high-duration-values" data-toc-modified-id="Checking-for-high-duration-values-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Checking for high duration values</a></span></li></ul></li><li><span><a href="#Drop" data-toc-modified-id="Drop-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Drop</a></span></li><li><span><a href="#Creating---weekday-and-weekend" data-toc-modified-id="Creating---weekday-and-weekend-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Creating - weekday and weekend</a></span></li><li><span><a href="#Understanding-the-difference-between-start_station-and-end_station" data-toc-modified-id="Understanding-the-difference-between-start_station-and-end_station-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Understanding the difference between start_station and end_station</a></span><ul class="toc-item"><li><span><a href="#Dropping-end_stations-outside-NY-and-system-warehouse" data-toc-modified-id="Dropping-end_stations-outside-NY-and-system-warehouse-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Dropping end_stations outside NY and system warehouse</a></span></li><li><span><a href="#Checking-again-the-difference" data-toc-modified-id="Checking-again-the-difference-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Checking again the difference</a></span></li></ul></li><li><span><a href="#Understanding-trips" data-toc-modified-id="Understanding-trips-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Understanding trips</a></span></li><li><span><a href="#Final-Dataframe---standardized-with-the-old-format" data-toc-modified-id="Final-Dataframe---standardized-with-the-old-format-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Final Dataframe - standardized with the old format</a></span></li><li><span><a href="#Creating-column-if-it's-holiday-or-not." data-toc-modified-id="Creating-column-if-it's-holiday-or-not.-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Creating column if it's holiday or not.</a></span></li></ul></div>

In [35]:
import pandas as pd
import numpy as np
import matplotlib
from tqdm import tqdm
import collections
import holidays

In [2]:
df = pd.read_csv(r'C:\Users\Raphael Costa\Desktop\Thiago\Data Mining\IronHack\Final Project Data Set\Citibike\new format\202102-202202.csv',
                dtype={"start_station_name" : 'category', 'start_station_id':'category', 'rideable_type':'category',
          'end_station_name' : 'category', 'end_station_id' : 'category', 'member_casual' : 'category'})

In [3]:
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28850536 entries, 0 to 28850535
Data columns (total 12 columns):
 #   Column              Dtype         
---  ------              -----         
 0   rideable_type       category      
 1   started_at          datetime64[ns]
 2   ended_at            datetime64[ns]
 3   start_station_name  category      
 4   start_station_id    category      
 5   end_station_name    category      
 6   end_station_id      category      
 7   start_lat           float64       
 8   start_lng           float64       
 9   end_lat             float64       
 10  end_lng             float64       
 11  member_casual       category      
dtypes: category(6), datetime64[ns](2), float64(4)
memory usage: 1.6 GB


# Creating duration in minutes column

In [5]:
## it worked, but with '.minute' it's going to show the difference in minutes
duration = []
for i in tqdm(range(0,len(df))):
    x = round(((df['ended_at'][i] - df['started_at'][i]).total_seconds())/60)
    if x<0:
        x = 60 + x
        duration.append(x)
    else:
        duration.append(x)
x = pd.DataFrame(duration)
df.insert(0, 'duration_min', x[0])

100%|██████████| 28850536/28850536 [19:07<00:00, 25142.31it/s]


In [6]:
test = df

## Checking for high duration values
- look like the duration with 25 hours or 1500 min is a trip canceled by the system.
- yeah!! It's confirmed.

In [7]:
df.loc[df['duration_min']==1500]

,duration_min,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
89,1500,docked_bike,2021-02-28 12:59:28,2021-03-01 13:59:08,E 16 St & Irving Pl,5938.11,W 15 St & 7 Ave,6030.06,40.735367,-73.987973,40.739355,-73.999318,casual
2885,1500,docked_bike,2021-02-17 18:05:55,2021-02-18 19:05:49,Broadway & W 51 St,6779.04,E 2 St & Avenue A,5553.10,40.762288,-73.983361,40.723077,-73.985836,casual
5323,1500,docked_bike,2021-02-03 22:16:51,2021-02-04 23:16:44,W 36 St & 9 Ave,6569.07,W 44 St & 5 Ave,6551.02,40.754623,-73.995167,40.755003,-73.980144,casual
10353,1500,docked_bike,2021-02-11 12:06:46,2021-02-12 13:06:37,E 32 St & Park Ave,6280.12,W 40 St & 5 Ave,6474.02,40.745712,-73.981948,40.752269,-73.982079,member
14124,1500,docked_bike,2021-02-14 21:33:33,2021-02-15 22:33:26,Lincoln Ave & E 138 St,7744.06,NaN,NaN,40.810893,-73.927311,NaN,NaN,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28270580,1500,classic_bike,2022-02-12 19:39:20,2022-02-13 20:39:15,Wythe Ave & Metropolitan Ave,5348.02,NaN,NaN,40.716887,-73.963198,NaN,NaN,casual
28270663,1500,classic_bike,2022-02-22 19:27:22,2022-02-23 20:27:17,West St & Chambers St,5329.03,NaN,NaN,40.717548,-74.013221,NaN,NaN,member
28270691,1500,classic_bike,2022-02-21 12:06:07,2022-02-22 13:05:47,Morris Ave & E Burnside Ave,8441.01,NaN,NaN,40.853220,-73.905460,NaN,NaN,casual
28270726,1500,classic_bike,2022-02-23 22:37:33,2022-02-24 23:37:28,2 Ave & E 72 St,6925.09,NaN,NaN,40.768762,-73.958408,NaN,NaN,member


In [65]:
df.loc[df['end_station_name'].isna()].loc[df['duration_min']==1500]

,duration_min,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
14124,1500,docked_bike,2021-02-14 21:33:33,2021-02-15 22:33:26,Lincoln Ave & E 138 St,7744.06,NaN,NaN,40.810893,-73.927311,NaN,NaN,casual
14211,1500,docked_bike,2021-02-25 17:32:07,2021-02-26 18:31:49,Pier 40 - Hudson River Park,5696.03,NaN,NaN,40.727714,-74.011295,NaN,NaN,casual
14295,1500,docked_bike,2021-02-18 16:23:36,2021-02-19 17:23:30,11 Ave & W 27 St,6425.04,NaN,NaN,40.751396,-74.005226,NaN,NaN,member
14369,1500,docked_bike,2021-02-26 12:23:15,2021-02-27 13:23:08,W 87 St & West End Ave,7484.05,NaN,NaN,40.789622,-73.977570,NaN,NaN,casual
14409,1500,docked_bike,2021-02-28 12:18:03,2021-03-01 13:17:53,South St & Gouverneur Ln,4953.04,NaN,NaN,40.703553,-74.006702,NaN,NaN,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28270479,1500,classic_bike,2022-02-09 20:59:34,2022-02-10 21:59:26,Stanton St & Norfolk St,5445.07,NaN,NaN,40.720747,-73.986274,NaN,NaN,casual
28270580,1500,classic_bike,2022-02-12 19:39:20,2022-02-13 20:39:15,Wythe Ave & Metropolitan Ave,5348.02,NaN,NaN,40.716887,-73.963198,NaN,NaN,casual
28270663,1500,classic_bike,2022-02-22 19:27:22,2022-02-23 20:27:17,West St & Chambers St,5329.03,NaN,NaN,40.717548,-74.013221,NaN,NaN,member
28270691,1500,classic_bike,2022-02-21 12:06:07,2022-02-22 13:05:47,Morris Ave & E Burnside Ave,8441.01,NaN,NaN,40.853220,-73.905460,NaN,NaN,casual


# Drop
- nulls - most of them are in 'end_station' info. I dropped them, 106k values in 28kk.
- rideable_type = 'electric' - after dropping the nulls, i found only 6 values for electrik bikes. Not worth it to keep, probably they are testing it.

In [12]:
test1 = df.dropna()

In [14]:
test1.drop(test1.loc[df['rideable_type']=='electric_bike'].index, inplace=True)

C:\Users\Raphael Costa\anaconda3\envs\class1\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [41]:
test1 = test1.reset_index(drop=True)

# Creating - weekday and weekend
- boolean 1,0 if it's weekday or weekend;

In [44]:
weekday = []
weekend = []
for i in tqdm(range(len(test1))):
    x = test1['started_at'][i].weekday()
    if x < 5:
        weekday.append('1')
        weekend.append('0')
    else:
        weekday.append('0')
        weekend.append('1')
print(collections.Counter(weekday))
print(collections.Counter(weekend))

100%|██████████| 28744392/28744392 [06:05<00:00, 78743.19it/s]


NameError: name 'collections' is not defined

In [50]:
test1['weekday'] = weekday

In [51]:
test1['weekend'] = weekend

In [62]:
test1.to_csv('test1.csv')

# Understanding the difference between start_station and end_station

In [3]:
%%time
test1 = pd.read_csv('test1.csv')

<decorator-gen-55>:2: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.


Wall time: 1min 30s


In [4]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28744392 entries, 0 to 28744391
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   duration_min        int64  
 2   rideable_type       object 
 3   started_at          object 
 4   ended_at            object 
 5   start_station_name  object 
 6   start_station_id    object 
 7   end_station_name    object 
 8   end_station_id      object 
 9   start_lat           float64
 10  start_lng           float64
 11  end_lat             float64
 12  end_lng             float64
 13  member_casual       object 
 14  weekday             int64  
 15  weekend             int64  
dtypes: float64(4), int64(4), object(8)
memory usage: 3.4+ GB


In [5]:
test1['start_station_id'].value_counts().count()

3169

In [6]:
test1['end_station_id'].value_counts().count()

3243

In [7]:
list_start_id = test1['start_station_id'].unique()

In [8]:
list_end_id = test1['end_station_id'].unique()

In [9]:
# list to check the difference between end and start station.
q = 0
list_not_common = []
for i in list_end_id:
    if i in list_start_id:
        q = +1
    else:
        list_not_common.append(i)        

In [10]:
list_not_common

['JC066',
 'SYS014',
 'JC098',
 'JC059',
 'JC008',
 'JC104',
 'JC057',
 'JC096',
 '7987.07',
 'JC013',
 'JC063',
 'JC009',
 'JC011',
 'JC002',
 'HB102',
 'HB502',
 'JC003',
 'JC072',
 'JC078',
 'JC094',
 'JC006',
 '8469.01',
 'JC053',
 'JC056',
 'HB203',
 '4953.01',
 'HB601',
 'JC035',
 'HB103',
 'JC052',
 'HB506',
 'JC102',
 'JC106',
 'HB507',
 'JC095',
 'HB302',
 'HB404',
 'JC005',
 'HB402',
 'JCSYS',
 'HB202',
 'HB602',
 'HB303',
 'HB403',
 'JC105',
 'JC051',
 'SYS037',
 'HB101',
 'JC024',
 'HB505',
 'JC103',
 'JC027',
 'JC038',
 'JC065',
 'HB603',
 'JC023',
 'HB304',
 'JC081',
 'JC082',
 'JC014',
 'HB401',
 'HB201',
 'JC055',
 'HB105',
 'JC018',
 'JC020',
 'JC077',
 'JC034',
 'JC019',
 'HB501',
 'JC099',
 'JC032',
 'JC076',
 'SYS030',
 'HB607',
 '8428.04',
 'SYS007']

 - These are the stations that are only in end station  IDs:
     - JC = New Jersey
     - SYS = system, warehouse
     - HB = Hoboken
     - JCSYS - New Jersey system
 - In this case the person get the bike in NY and drop it in another city. And for the system, probably it's a mechanical issue, so the company gets the bike to fix it. 
 - I'm going to drop them.

## Dropping end_stations outside NY and system warehouse

In [11]:
test2 = test1

In [12]:
# getting the index to drop
list_index_drop = []
for i in tqdm(list_not_common):
    index = test2.loc[test2['end_station_id']==i].index.values
    for x in range(len(index)):
        list_index_drop.append(index[x])

100%|██████████| 77/77 [01:35<00:00,  1.24s/it]


In [13]:
len(list_index_drop)

1830

In [15]:
%%time
indexes_to_keep = set(range(test1.shape[0])) - set(list_index_drop)
test2 = test1.take(list(indexes_to_keep))

Wall time: 6min 59s
Compiler : 594 ms
Parser   : 420 ms


## Checking again the difference

In [16]:
list_start_id = test2['start_station_id'].unique()
list_end_id = test2['end_station_id'].unique()

In [17]:
len(list_start_id)

3169

In [18]:
len(list_end_id)

3166

In [19]:
# list to check the difference between end and start station.
q = 0
list_not_common = []
for i in list_start_id:
    if i in list_end_id:
        q = +1
    else:
        list_not_common.append(i)

In [20]:
# the same station name has different id's. So probably they changed the id of these stations. Still real locations...
list_not_common

[6551.03, 3038.08, 8428.04]

In [21]:
test2.loc[test2['start_station_id']== list_not_common[0]]

,Unnamed: 0,duration_min,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,weekday,weekend
6807,6807,66,docked_bike,2021-02-13 10:16:14,2021-02-13 11:22:44,E 43 St & 5 Ave,6551.03,W 44 St & 5 Ave,6551.02,40.754120,-73.980251,40.755003,-73.980144,member,0,1
6811,6811,5,docked_bike,2021-02-11 08:12:24,2021-02-11 08:16:56,E 43 St & 5 Ave,6551.03,Broadway & W 29 St,6289.06,40.754120,-73.980251,40.746201,-73.988557,member,1,0
10739,10739,2,docked_bike,2021-02-08 19:05:31,2021-02-08 19:07:14,E 43 St & 5 Ave,6551.03,W 40 St & 5 Ave,6474.02,40.754121,-73.980252,40.752269,-73.982079,member,1,0
10930,10930,10,docked_bike,2021-02-11 10:46:34,2021-02-11 10:56:27,E 43 St & 5 Ave,6551.03,W 20 St & 7 Ave,6182.02,40.754120,-73.980251,40.742388,-73.997262,member,1,0
11563,11563,111,docked_bike,2021-02-14 14:33:52,2021-02-14 16:24:27,E 43 St & 5 Ave,6551.03,Broadway & Battery Pl,4962.01,40.754120,-73.980251,40.704633,-74.013617,casual,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628795,628795,32,docked_bike,2021-02-10 17:30:41,2021-02-10 18:03:07,E 43 St & 5 Ave,6551.03,Riverside Dr & W 82 St,7388.10,40.754120,-73.980251,40.787209,-73.981281,member,1,0
639959,639959,12,docked_bike,2021-02-08 19:13:26,2021-02-08 19:25:10,E 43 St & 5 Ave,6551.03,E 15 St & 3 Ave,5863.07,40.754120,-73.980251,40.734232,-73.986923,member,1,0
639981,639981,32,docked_bike,2021-02-10 17:26:17,2021-02-10 17:58:39,E 43 St & 5 Ave,6551.03,E 17 St & Broadway,5980.07,40.754120,-73.980251,40.737050,-73.990093,member,1,0
639982,639982,27,docked_bike,2021-02-10 12:21:04,2021-02-10 12:48:09,E 43 St & 5 Ave,6551.03,E 17 St & Broadway,5980.07,40.754120,-73.980251,40.737050,-73.990093,member,1,0


# Understanding trips

In [22]:
%%time
unique_start_end = test2[['start_station_id','end_station_id']].nunique()

Wall time: 4.9 s


In [23]:
# theorical number of possible unique trips
unique_start_end.agg('product')

10033054

In [24]:
# actual number of unique trips
test2[['start_station_name','end_station_name']].value_counts().count()

876533

# Final Dataframe - standardized with the old format
- drop one of the weekends (if it's 0 on weekdays, then it's weekend);

In [25]:
test2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28742562 entries, 0 to 28744391
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   duration_min        int64  
 2   rideable_type       object 
 3   started_at          object 
 4   ended_at            object 
 5   start_station_name  object 
 6   start_station_id    object 
 7   end_station_name    object 
 8   end_station_id      object 
 9   start_lat           float64
 10  start_lng           float64
 11  end_lat             float64
 12  end_lng             float64
 13  member_casual       object 
 14  weekday             int64  
 15  weekend             int64  
dtypes: float64(4), int64(4), object(8)
memory usage: 3.6+ GB


In [26]:
%%time
df_final = test2[['duration_min','started_at','ended_at','start_station_name','start_station_id','end_station_name','end_station_id','start_lat','start_lng','end_lat','end_lng','member_casual','weekday','weekend']]

Wall time: 19.7 s


In [27]:
new_colnames = ('tripduration','started_at','ended_at','start_station_name','start_station_id','end_station_name','end_station_id','start_lat','start_lng','end_lat','end_lng','member_casual','weekday','weekend')

In [28]:
%%time
df_final.columns = new_colnames

Wall time: 296 ms


In [29]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28742562 entries, 0 to 28744391
Data columns (total 14 columns):
 #   Column              Dtype  
---  ------              -----  
 0   tripduration        int64  
 1   started_at          object 
 2   ended_at            object 
 3   start_station_name  object 
 4   start_station_id    object 
 5   end_station_name    object 
 6   end_station_id      object 
 7   start_lat           float64
 8   start_lng           float64
 9   end_lat             float64
 10  end_lng             float64
 11  member_casual       object 
 12  weekday             int64  
 13  weekend             int64  
dtypes: float64(4), int64(3), object(7)
memory usage: 3.2+ GB


# Creating column if it's holiday or not.

In [32]:
df_final = df_final.drop('weekend',axis=1)

In [42]:
df_final = df_final.reset_index(drop=True)

In [43]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28742562 entries, 0 to 28742561
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   tripduration        int64  
 1   started_at          object 
 2   ended_at            object 
 3   start_station_name  object 
 4   start_station_id    object 
 5   end_station_name    object 
 6   end_station_id      object 
 7   start_lat           float64
 8   start_lng           float64
 9   end_lat             float64
 10  end_lng             float64
 11  member_casual       object 
 12  weekday             int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 2.8+ GB


In [37]:
holiday_NY = holidays.US(state='NY')

In [44]:
holiday = []
for i in tqdm(range(len(df_final))):
    if df_final['started_at'][i] in holiday_NY:
        holiday.append('1')
    else:
        holiday.append('0')
        

100%|██████████| 28742562/28742562 [30:40<00:00, 15617.82it/s]


In [46]:
%%time
df_final['holiday'] = holiday

Wall time: 528 ms


In [59]:
%%time
df_final.to_csv(r'C:\Users\Raphael Costa\Desktop\Thiago\Data Mining\IronHack\Final Project Data Set\Citibike\final_database\2021-2022.csv', index = False)

Wall time: 15min 2s
